In [1]:
from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize #Used to extract words from documents
from nltk.stem import WordNetLemmatizer #Used to lemmatize words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans

import sys
from time import time

import pandas as pd
import numpy as np


import tensorflow_hub as hub
from sklearn.cluster import KMeans
import time
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics.pairwise import cosine_similarity

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [14]:
data1 = pd.read_csv('/kaggle/input/films-classification/train.csv')

In [4]:
data

,name,genre,text
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...
...,...,...,...
54209,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on ...
54210,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The sist...
54211,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about g..."
54212,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and hav...


In [25]:
data

,name,genre,text
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...
...,...,...,...
9995,"""Friday Night Cranks"" (2007/I)",reality-tv,The show began with Jared and Kristen in 2007...
9996,Donkey in the City: 3D Animation Film (????),animation,The center characters of the film are two kid...
9997,Ashland (2007),drama,"Ashland is the story of Ash Slavin, a lonely ..."
9998,Whispering Smith (1926),western,Railroad foreman Murray Sinclair is dismissed...


In [26]:
from sentence_transformers import SentenceTransformer
model_path = "paraphrase-distilroberta-base-v1"
model = SentenceTransformer(model_path)
corpus_embeddings = model.encode(data.text)
from sklearn.cluster import KMeans
import scipy
kmeans = KMeans(n_clusters=10,random_state=0).fit(corpus_embeddings)
cls_dist=pd.Series(kmeans.labels_).value_counts()

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [29]:
idx = 54209
text = data1.loc[idx, "text"]
print(data1.loc[idx]['name'])

text_embedding = model.encode([text])


cluster = kmeans.predict(text_embedding)[0]
cluster_embeddings = corpus_embeddings[kmeans.labels_ == cluster]
similarities = cosine_similarity(text_embedding, cluster_embeddings)
top_indices = similarities.argsort()[0][-10:]
top_neighbors = data.iloc[kmeans.labels_ == cluster].iloc[top_indices]

print(top_neighbors)
%time
# Ваш код для замера времени исполнения


 "Bonino" (1953) 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                             name        genre  \
6016                             Cactuses (2006)         drama   
7725   Missing Allen - Wo ist Allen Ross? (2002)   documentary   
2321                      Sincerely Yours (1955)         drama   
3133                   Symphony of Living (1935)         drama   
835                  "The Great Zucchini" (2013)        comedy   
4604                         Dead Air (2008/III)      thriller   
7194                            9th & Bay (2009)         drama   
3229       Marty Robbins: Music Anthology (2008)         music   
8117                          Habana muda (2011)   documentary   
16                          Babylon Vista (2001)        comedy   

                                                   text  
6016   Simon Brodigan gives up on just about everyth...  
7725   When beloved Chicago movie cameraman Allen Ro...  
2321   Tony Warrin has it all: a popular pianist who...  
3133   There is this old concertmaster of

In [30]:
from sentence_transformers import SentenceTransformer
model_path = "paraphrase-MiniLM-L6-v2"
model = SentenceTransformer(model_path)
corpus_embeddings = model.encode(data.text)
from sklearn.cluster import KMeans
import scipy
kmeans = KMeans(n_clusters=10,random_state=0).fit(corpus_embeddings)
cls_dist=pd.Series(kmeans.labels_).value_counts()

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [33]:
idx = 54209
text = data1.loc[idx, "text"]
print(data1.loc[idx]['name'])

text_embedding = model.encode([text])


cluster = kmeans.predict(text_embedding)[0]
cluster_embeddings = corpus_embeddings[kmeans.labels_ == cluster]
similarities = cosine_similarity(text_embedding, cluster_embeddings)
top_indices = similarities.argsort()[0][-10:]
top_neighbors = data.iloc[kmeans.labels_ == cluster].iloc[top_indices]

print(top_neighbors)
%time
# Ваш код для замера времени исполнения


 "Bonino" (1953) 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                              name        genre  \
378                             Fight Club (1999)         drama   
1667   Darker Secrets: Sideline Secrets II (2008)         drama   
7594                       Quentin Amar Li (1999)        action   
6294                          Matok Ve'Mar (2007)         drama   
5041                    "All Together Now" (2003)        comedy   
7144                             Wannabe (2005/I)        comedy   
7730                        Run Ronnie Run (2002)        comedy   
1504                                Bookie (2017)        comedy   
9819                  She Couldn't Take It (1935)        comedy   
1884                        Resident Alien (1990)   documentary   

                                                   text  
378    A nameless first person narrator ('Edward Nor...  
1667   Peter, a young traffic cop, is asked to look ...  
7594   All that Daniel ever wanted to do was to make...  
6294   Ran directs a successfu

In [34]:
from sentence_transformers import SentenceTransformer
model_path = "stsb-roberta-base-v2"
model = SentenceTransformer(model_path)
corpus_embeddings = model.encode(data.text)
from sklearn.cluster import KMeans
import scipy
kmeans = KMeans(n_clusters=10,random_state=0).fit(corpus_embeddings)
cls_dist=pd.Series(kmeans.labels_).value_counts()

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [35]:
idx = 54209
text = data1.loc[idx, "text"]
print(data1.loc[idx]['name'])

text_embedding = model.encode([text])


cluster = kmeans.predict(text_embedding)[0]
cluster_embeddings = corpus_embeddings[kmeans.labels_ == cluster]
similarities = cosine_similarity(text_embedding, cluster_embeddings)
top_indices = similarities.argsort()[0][-10:]
top_neighbors = data.iloc[kmeans.labels_ == cluster].iloc[top_indices]

print(top_neighbors)
%time
# Ваш код для замера времени исполнения

 "Bonino" (1953) 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                  name        genre  \
8617   Danny and the Human Zoo (2015)         drama   
9195     Rafi Bukai, Aba Sheli (2012)   documentary   
4064              Jungle Story (1996)         drama   
9152    Soshite chichi ni naru (2013)         drama   
2920          Five Easy Pieces (1970)         drama   
9943         In from the Night (2006)         drama   
7586                    Lifted (2010)         drama   
3547         The Yellow Canary (1963)         drama   
4977              "Hey Dad..!" (1987)        comedy   
16               Babylon Vista (2001)        comedy   

                                                   text  
8617   In 1974 in Dudley in the English Midlands bla...  
9195   Guy Bukaee was two years and four months old ...  
4064   A fictitious account of the early years of th...  
9152   Would you choose your natural son, or the son...  
2920   Robert Dupea has given up his promising caree...  
9943   After ten years without contact, Bobby 

In [36]:
from sentence_transformers import SentenceTransformer
model_path = "distiluse-base-multilingual-cased-v2"
model = SentenceTransformer(model_path)
corpus_embeddings = model.encode(data.text)
from sklearn.cluster import KMeans
import scipy
kmeans = KMeans(n_clusters=10,random_state=0).fit(corpus_embeddings)
cls_dist=pd.Series(kmeans.labels_).value_counts()

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [38]:
idx = 54209
text = data1.loc[idx, "text"]
print(data1.loc[idx]['name'])

text_embedding = model.encode([text])


cluster = kmeans.predict(text_embedding)[0]
cluster_embeddings = corpus_embeddings[kmeans.labels_ == cluster]
similarities = cosine_similarity(text_embedding, cluster_embeddings)
top_indices = similarities.argsort()[0][-10:]
top_neighbors = data.iloc[kmeans.labels_ == cluster].iloc[top_indices]

print(top_neighbors)
%time
# Ваш код для замера времени исполнения

 "Bonino" (1953) 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                                   name        genre  \
9704                               Live in Reno (2004)         music   
5680   Perfect Bid: The Contestant Who Knew Too Much...  documentary   
6140                             "Johnnie Talk" (2003)        comedy   
3713   Kendall Ross Bean: Chopin Polonaise in A Flat...        music   
4365   The Steve-O Video: Vol. II - The Tour Video (...       comedy   
6578                          Square Dance Katy (1950)       musical   
1916                  Tiny Tim: The Last Hurrah (2005)   documentary   
4024                               Rocanrol '68 (2013)        comedy   
6460                           Pop Punk Zombies (2011)        horror   
7730                             Run Ronnie Run (2002)        comedy   

                                                   text  
9704   The Orchestra/Live in Reno is a concert film ...  
5680   Winner of "Best Documentary" at the Orlando F...  
6140   On Johnnie Talk, our host 

In [39]:
from sentence_transformers import SentenceTransformer
model_path = "bert-base-nli-mean-tokens"
model = SentenceTransformer(model_path)
corpus_embeddings = model.encode(data.text)
from sklearn.cluster import KMeans
import scipy
kmeans = KMeans(n_clusters=10,random_state=0).fit(corpus_embeddings)
cls_dist=pd.Series(kmeans.labels_).value_counts()

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [40]:
idx = 54209
text = data1.loc[idx, "text"]
print(data1.loc[idx]['name'])

text_embedding = model.encode([text])


cluster = kmeans.predict(text_embedding)[0]
cluster_embeddings = corpus_embeddings[kmeans.labels_ == cluster]
similarities = cosine_similarity(text_embedding, cluster_embeddings)
top_indices = similarities.argsort()[0][-10:]
top_neighbors = data.iloc[kmeans.labels_ == cluster].iloc[top_indices]

print(top_neighbors)
%time
# Ваш код для замера времени исполнения

 "Bonino" (1953) 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                name  genre  \
2649      Looking for Sunday (2006)   drama   
8808        Come Next Spring (1956)   drama   
6194         Unknown Soldier (2004)   drama   
8943           De ooggetuige (1995)   drama   
9943       In from the Night (2006)   drama   
4167                   Bingo (1974)   drama   
4061      American Dream (2008/III)   short   
2920        Five Easy Pieces (1970)   drama   
0      Oscar et la dame rose (2009)   drama   
9376                 Azzurro (2000)   drama   

                                                   text  
2649   Like many young men in their mid-twenties, tw...  
8808   Matt Ballot has returned home after 12 years ...  
6194   Ellison (aka L) just graduated high school an...  
8943   Paul was blinded in an accident when he was f...  
9943   After ten years without contact, Bobby Miller...  
4167   Photography student François leads a mostly c...  
4061   Miguel lived a very simple happy life with hi...  
2920   Robert Dupe

In [41]:
from sentence_transformers import SentenceTransformer
model_path = "bert-base-nli-mean-tokens"
model = SentenceTransformer(model_path)
corpus_embeddings = model.encode(data.text)
from sklearn.cluster import KMeans
import scipy
kmeans = KMeans(n_clusters=30,random_state=0).fit(corpus_embeddings)
cls_dist=pd.Series(kmeans.labels_).value_counts()

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [43]:
idx = 54209
text = data1.loc[idx, "text"]
print(data1.loc[idx]['name'])

text_embedding = model.encode([text])


cluster = kmeans.predict(text_embedding)[0]
cluster_embeddings = corpus_embeddings[kmeans.labels_ == cluster]
similarities = cosine_similarity(text_embedding, cluster_embeddings)
top_indices = similarities.argsort()[0][-10:]
top_neighbors = data.iloc[kmeans.labels_ == cluster].iloc[top_indices]

print(top_neighbors)
%time
# Ваш код для замера времени исполнения

 "Bonino" (1953) 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                             name   genre  \
6194      Unknown Soldier (2004)    drama   
9878      Le bel été 1914 (1996)    drama   
7586               Lifted (2010)    drama   
8943        De ooggetuige (1995)    drama   
4167                Bingo (1974)    drama   
4061   American Dream (2008/III)    short   
492            The Limits (2007)    drama   
2920     Five Easy Pieces (1970)    drama   
9376              Azzurro (2000)    drama   
4977         "Hey Dad..!" (1987)   comedy   

                                                   text  
6194   Ellison (aka L) just graduated high school an...  
9878   A couple take their son for the holidays to a...  
7586   Henry Matthews is an exceptionally talented y...  
8943   Paul was blinded in an accident when he was f...  
4167   Photography student François leads a mostly c...  
4061   Miguel lived a very simple happy life with hi...  
492    Sanjeet is an Electrical Engineer from Bombay...  
2920   Robert Dupea has given up his pro

In [44]:
from sentence_transformers import SentenceTransformer
model_path = "bert-base-nli-mean-tokens"
model = SentenceTransformer(model_path)
corpus_embeddings = model.encode(data.text)
from sklearn.cluster import KMeans
import scipy
kmeans = KMeans(n_clusters=5,random_state=0).fit(corpus_embeddings)
cls_dist=pd.Series(kmeans.labels_).value_counts()

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [45]:
idx = 54209
text = data1.loc[idx, "text"]
print(data1.loc[idx]['name'])

text_embedding = model.encode([text])


cluster = kmeans.predict(text_embedding)[0]
cluster_embeddings = corpus_embeddings[kmeans.labels_ == cluster]
similarities = cosine_similarity(text_embedding, cluster_embeddings)
top_indices = similarities.argsort()[0][-10:]
top_neighbors = data.iloc[kmeans.labels_ == cluster].iloc[top_indices]

print(top_neighbors)
%time
# Ваш код для замера времени исполнения

 "Bonino" (1953) 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                                name   genre  \
3640                     Capture the Flag (2010/II)    short   
2920                        Five Easy Pieces (1970)    drama   
0                      Oscar et la dame rose (2009)    drama   
3133                      Symphony of Living (1935)    drama   
5621             How to Make Love to a Woman (2010)   comedy   
6660                          I'm Losing You (1998)    drama   
9376                                 Azzurro (2000)    drama   
4977                            "Hey Dad..!" (1987)   comedy   
8429                               Santa Con (2014)   comedy   
8737   National Lampoon's Christmas Vacation (1989)   comedy   

                                                   text  
3640   Adapted from the novel by Rebecca Chace, Capt...  
2920   Robert Dupea has given up his promising caree...  
0      Listening in to a conversation between his do...  
3133   There is this old concertmaster of the Cosmop...  
5621 